In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

In [10]:
weekend_Jan_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]

In [11]:
link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in weekend_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jan_list)

25920

In [13]:
x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 1080)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
x[:,:2]

matrix([[ 1295.70655466,  1301.84667963],
        [ 1260.15802585,  1230.47158613],
        [ 1583.42086625,  1571.18058951],
        [ 1542.85786168,  1540.42855181],
        [ 1603.88435238,  1605.71175094],
        [ 1648.49635808,  1637.72762733],
        [ 1355.5558366 ,  1347.85549439],
        [ 1338.46391474,  1332.71811576],
        [ 1687.78738992,  1704.94063122],
        [ 1657.04422278,  1672.71186971],
        [ 1415.94813912,  1517.41559903],
        [ 1273.01843746,  1239.54193386],
        [ 1259.05925946,  1241.80419088],
        [ 1333.82238428,  1378.52477044],
        [ 1309.99036988,  1185.31930605],
        [ 1036.31942253,  1049.95646759],
        [ 1308.58598201,  1258.72365619],
        [ 1333.8562281 ,  1328.36982322],
        [ 1474.43840622,  1524.14315656],
        [ 1494.06320587,  1562.19011815],
        [ 1381.21486359,  1426.83485745],
        [ 1375.10558197,  1395.27194191],
        [ 1296.61363339,  1269.32283407],
        [ 1339.57767211,  1329.347

In [15]:
np.size(x, 0), np.size(x, 1)

(24, 962)

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e+01, 2e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 26
 AA' NZ     : 3.250e+02
 Factor NZ  : 3.510e+02
 Factor Ops : 6.201e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -6.53173041e+06  0.00000000e+00  5.51e+03 8.57e+02  1.00e+06     0s
   1  -4.02264898e+06 -3.53732054e+06  1.26e+03 2.26e+02  2.75e+05     0s
   2  -3.05641575e+06 -4.57200471e+06  2.65e+01 4.51e+00  3.21e+04     0s
   3  -3.50306997e+06 -3.73021173e+06  2.65e-05 4.12e-06  4.0

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jan_weekend.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[4.791264514764715e-11,
 849.2083931426245,
 5.3595434465588924e-11,
 1283.175682750575,
 168.01685394262827,
 481.69172441802675,
 1.6173437414168743e-08,
 0.00032048772966450584,
 1.4294174561247812e-10,
 1377.3098690922338,
 1.4294174561022437e-10,
 1.4294174561085385e-10,
 1.429417456081637e-10,
 1.4376143803429263e-10,
 1682.4843948260566,
 16.24159119779332,
 0.3488986243236226,
 14.897207022512092,
 1151.8320261960323,
 4.15014804560154,
 0.6740520025047196,
 748.860574765309,
 772.8776184233674,
 3.985515674155348,
 608.3817493379187,
 497.1935079797559,
 2.93992133073413,
 1308.906166207192,
 0.4557202840123191,
 0.19667675221262165,
 249.82996757064504,
 1300.2302849428306,
 865.0445701043892,
 1274.4348623865358,
 0.500818357143409,
 2.3978204949555577e-05,
 2.397911321363507e-05,
 2.397745113832422e-05,
 0.4031815269820016,
 988.6452197117513,
 771.8004404200396,
 533.8601831196049,
 333.1373142031732,
 850.4270061092483,
 87.38263539847941,
 0.34522208264359666,
 290.10544